In [9]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import glob
import io
from collections import namedtuple
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

### Set FilePath

In [10]:
LABEL_PATH = r"F:\Minor Project\Monument Object Detection\helper\Report\all_there_is.csv"
ignore_list = ['kotilingeshvara', 'til mahadev narayan temple', 'king statue']
mistake_list = ['degutale temple', 'kritipur tower']
correct_list = ['degu tale temple_KDS', 'kirtipur tower', 'degu tale']

In [11]:
POSSIBLE_LABELS = dict()

In [12]:
def csv_to_label_map_dict(path):
    df = pd.read_csv(path)
    class_list = list(df['class'])
    final_class_list = list()
    for class_name in class_list:
        if class_name not in ignore_list and class_name not in correct_list:
            if class_name in mistake_list:
                if class_name == 'degutale temple' or class_name == 'degu tale':
                    class_name = correct_list[0]
                elif class_name == 'kritipur tower':
                    class_name = correct_list[1]
            final_class_list.append(class_name)
    class_index = 1
    res_dict = {}
    for class_label in final_class_list:
        res_dict[class_label] = class_index
        class_index += 1
    return res_dict

In [13]:
csv_to_label_map_dict(LABEL_PATH)

{'badrinath temple': 1,
 'basantapur tower': 2,
 'bhagavati temple': 3,
 'bhairavnath temple': 4,
 'bhaktapur tower': 5,
 'bhimeleshvara': 6,
 'bhimsen temple': 7,
 'bhupatindra malla column': 8,
 'bhuvana lakshmeshvara': 9,
 'chasin dega': 10,
 'chayasilin mandap': 11,
 'dattatreya temple': 12,
 'degu tale temple_KDS': 13,
 'fasidega temple': 14,
 'gaddi durbar': 15,
 'garud': 16,
 'golden gate': 17,
 'gopinath krishna temple': 18,
 'hanuman idol': 19,
 'indrapura': 20,
 'jagannatha temple': 21,
 'kala-bhairava': 22,
 'kasthamandap': 23,
 'kavindrapura sattal': 24,
 'kedamatha tirtha': 25,
 'kirtipur tower': 26,
 'kumari ghar': 27,
 'lalitpur tower': 28,
 'mahadev temple': 29,
 'narayan temple': 30,
 'national gallery': 31,
 'nyatapola temple': 32,
 'palace of the 55 windows': 33,
 'panchamukhi hanuman': 34,
 'pratap malla column': 35,
 'shiva temple': 36,
 'shveta bhairava': 37,
 'siddhi lakshmi temple': 38,
 'simha sattal': 39,
 'taleju bell_BDS': 40,
 'taleju bell_KDS': 41,
 'talej

In [14]:
POSSIBLE_LABELS = csv_to_label_map_dict(LABEL_PATH)

In [15]:
POSSIBLE_LABELS['bg'] = 0

In [17]:
def check_class_validity(class_name):
    if class_name == 'degutale temple' or class_name == 'degu tale':
        return 'degu tale temple_KDS'
    elif class_name == 'kritipur tower':
        return 'kirtipur tower'
    elif class_name not in list(POSSIBLE_LABELS.keys()):
        return None
    else:
        return class_name

In [18]:
class_list = []
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        all_objects = root.findall('object')
        changed_filename = xml_file.split('\\')[-1].split('.')[0] + '.JPG'
        # Handling xml parser for random background images
        if len(all_objects) != 0:
            for member in all_objects:
                class_label = check_class_validity(member[0].text)
                if class_label is not None:
                    value = (changed_filename,
                             int(root.find('size')[0].text),
                             int(root.find('size')[1].text),
                             class_label,
                             int(member[4][0].text),
                             int(member[4][1].text),
                             int(member[4][2].text),
                             int(member[4][3].text)
                             )
                    if class_label not in class_list:
                        class_list.append(class_label)
                    xml_list.append(value)
        else:
            # Setting background class to 'bg', and four bbox coordinates all to neg 1.
            value = (changed_filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     'bg',
                      -1,
                      -1,
                      -1,
                      -1)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


In [19]:
# xml_to_csv(r"F:\Minor Data Collection\Final Image Data\Monument Original\Merged_Original_Augmented\test\Annotations")

In [20]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [21]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [22]:
def class_text_to_int(row_label):
    return POSSIBLE_LABELS[row_label]

In [23]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        if int(row['xmin']) > -1:
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
            
        else:
            xmins.append(0)
            xmaxs.append(0)
            ymins.append(0)
            ymaxs.append(0)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(0)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example

In [24]:
def TFRecord_Creator(image_dir, annotations_dir, output_file, split_type = "BDG"):
    writer = tf.io.TFRecordWriter(output_file)
    path = os.path.join(image_dir)
    examples = xml_to_csv(annotations_dir)
    #grouped by the filename so that multiple monument detection boxes are grouped in a single 'GROUP'
    grouped = split(examples, 'filename') 
    for group in tqdm(grouped):
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file for ', split_type)

In [25]:
BASE_DIR = r"F:\Minor Data Collection\Final Image Data\Monument Original\Augmented Image Dataset (2nd try)\Splitted"
annotations_path = 'Annotations'
jpg_images_path = 'JPEGImages'
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

In [27]:
TFRecord_Creator(os.path.join(train_dir, jpg_images_path, "images"), os.path.join(train_dir, annotations_path), os.path.join(BASE_DIR, 'train_aug_v3_random.tfrecord'))

# TFRecord_Creator(os.path.join(validation_dir, jpg_images_path), os.path.join(validation_dir, annotations_path), os.path.join(BASE_DIR, 'validation_ori.tfrecord'))

# TFRecord_Creator(os.path.join(test_dir, jpg_images_path), os.path.join(test_dir, annotations_path), os.path.join(BASE_DIR, 'test_ori.tfrecord'))

2it [00:00, 1996.34it/s]
1it [00:00, 1003.66it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.98it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
2it [00:00, 999.71it/s]
3it [00:00, 3005.23it/s]
2it [00:00, 501.35it/s]
2it [00:00, ?it/s]
2it [00:00, 2006.84it/s]
2it [00:00, 2004.93it/s]
1it [00:00, 1021.51it/s]
1it [00:00, ?it/s]
1it [00:00, 1021.75it/s]
1it [00:00, 1002.94it/s]
2it [00:00, 1999.19it/s]
2it [00:00, 1991.12it/s]
2it [00:00, ?it/s]
2it [00:00, 2001.10it/s]
3it [00:00, ?it/s]
3it [00:00, 3005.23it/s]
3it [00:00, 3009.55it/s]
3it [00:00, ?it/s]
2it [00:00, 2006.36it/s]
2it [00:00, 1996.81it/s]
2it [00:00, ?it/s]
2it [00:00, 2004.93it/s]
2it [00:00, 1921.35it/s]
2it [00:00, ?it/s]
2it [00:00, 2010.21it/s]
2it [00:00, ?it/s]
1it [00:00, 995.80it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2005.88it/s]
1it [00:00, 334.10it/s]
2it [00:00, 2003.97it/s]
2it [00:00, ?it/s]
2it [00:00, 2004.93it/s]
2it [00:00, 2023.79it/s]
4it [00:00, 4

1it [00:00, ?it/s]
3it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1041.29it/s]
1it [00:00, 1048.58it/s]
1it [00:00, ?it/s]
1it [00:00, 1042.84it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
2it [00:00, 2010.21it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
6it [00:00, 6010.47it/s]
1it [00:00, ?it/s]
3it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1003.42it/s]
4it [00:00, ?it/s]
1it [00:00, ?it/s]
4it [00:00, 4007.94it/s]
2it [00:00, ?it/s]
1it [00:00, 963.76it/s]
3it [00:00, 3000.93it/s]
1it [00:00, 962.66it/s]
1it [00:00, 1003.42it/s]
2it [00:00, 2085.68it/s]
2it [00:00, 2004.93it/s]
2it [00:00, ?it/s]
3it [00:00, ?it/s]
3it [00:00, 3004.52it/s]
3it [00:00, 3003.08it/s]
3it [00:00, ?it/s]
1it [00:00, 1052.26it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 2002.53it/s]
5it [00:00, ?it/s]
5it [00:00, ?it/s]
5it [00:00, 5130.02it/s]
2it [00:00, 2002.05it/s]
1it [00:00, ?it

1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, 3008.11it/s]
3it [00:00, 3018.21it/s]
3it [00:00, 3005.95it/s]
3it [00:00, 3004.52it/s]
1it [00:00, 999.83it/s]
1it [00:00, 966.43it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1037.17it/s]
4it [00:00, 4018.49it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1036.91it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1058.90it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1020.76it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.38it/s]
1it [00:00, 1005.11it/s]
1it [00:00, ?it/s]
1it [00:00, 1033.84it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
1it [00:00, 967.99it/s]
1it [00:00, ?it/s]
1it [00:00, 968.21it/s]
1it [00:00, 965.10it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
4it [00:00, 3875.54it/s]
5it [00:00, 4995.60it/s]
2it [00:00, ?it/s]
3it [00:00, ?it/s]
2it [00:00, 2040.03

1it [00:00, 1002.22it/s]
1it [00:00, 997.93it/s]
1it [00:00, 1016.55it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, 1008.00it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 997.69it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.62it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 501.11it/s]
1it [00:00, 1026.00it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 1010.19it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 143.08it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1000.31it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1005.11it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 501.35it/s]
1it [00:00, 1005.35it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1000.79it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1004.38it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 499.44it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
1it [00:00, 1005.35it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
1it [00:00,

1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.38it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1006.79it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1004.62it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1007.76it/s]
1it [00:00, 1008.00it/s]
1it [00:00, 1002.94it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.03it/s]
1it [00:00, ?it/s]
1it [00:00, 1008.97it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 501.35it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1005.35it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 502.97it/s]
1it [00:00, 1002.94it/s]
1it [0

...

2it [00:00, ?it/s]
2it [00:00, 2008.77it/s]
2it [00:00, ?it/s]
2it [00:00, 2003.49it/s]
2it [00:00, ?it/s]
2it [00:00, 1963.63it/s]
2it [00:00, 1002.10it/s]
2it [00:00, 1998.24it/s]
2it [00:00, 2000.14it/s]
2it [00:00, 1003.42it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 1003.66it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2007.32it/s]
2it [00:00, 2003.01it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 1001.86it/s]
2it [00:00, 2003.97it/s]
2it [00:00, 2002.53it/s]
2it [00:00, 2008.29it/s]
2it [00:00, 2004.93it/s]
2it [00:00, 2002.05it/s]
2it [00:00, 2006.84it/s]
2it [00:00, 2004.45it/s]
2it [00:00, 2006.84it/s]
2it [00:00, 2004.93it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 1935.53it/s]
2it [00:00, 2080.51it/s]
2it [00:00, 2008.77it/s]
2it [00:00, 2027.70it/s]
2it [00:00, 2028.68it/s]
2it [00:00, 2119.94it/s]
2it [00:00, 2003.97it/s]
2it [00:00, 1935.98it/s]
2it [00:00, ?it/s]
2it [00:00, 2001.10it/s]
2it [00:00, 2002.53it/s]
2it [00:00, 2081.54it/s]
2it [00:00,

1it [00:00, 1001.98it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1003.42it/s]
2it [00:00, 2002.53it/s]
2it [00:00, 2001.10it/s]
2it [00:00, 2004.45it/s]
3it [00:00, 3007.39it/s]
3it [00:00, 3010.99it/s]
3it [00:00, 1501.00it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2004.45it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
2it [00:00, 2006.36it/s]
1it [00:00, ?it/s]
1it [00:00, 993.68it/s]
1it [00:00, 1000.79it/s]
1it [00:00, 500.75it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 1002.46it/s]
2it [00:00, 2007.32it/s]
3it [00:00, 3008.11it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1008.97it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.27it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.55it/s]
3it [00:00, 3005.95it/s]
2it [00:00, 2008.77it/s]
2it [00:00, 2003.97it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1008.25it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.27it/s]

1it [00:00, 1001.27it/s]
1it [00:00, 1007.28it/s]
2it [00:00, 2004.93it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
2it [00:00, 2007.80it/s]
2it [00:00, 2006.36it/s]
2it [00:00, 2004.93it/s]
2it [00:00, ?it/s]
2it [00:00, 2003.01it/s]
1it [00:00, 499.38it/s]
1it [00:00, ?it/s]
2it [00:00, 999.60it/s]
2it [00:00, 1003.42it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 1001.39it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1004.38it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
2it [00:00, 2008.77it/s]
1it [00:00, 1010.19it/s]
1it [00:00, 1008.25it/s]
1it [00:00, 1012.14it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
4it [00:00, 4035.90it/s]
4it [00:00, 2005.88it/s]
4it [00:00, 2010.69it/s]
1it [00:00, 1001.03it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.90it/s]
1it [00:00, 996.75it/s]
1it [00:00, 1002.94it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1000.79it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.27it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [0

1it [00:00, 1002.46it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 1010.43it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1001.74it/s]
1it [00:00, ?it/s]
2it [00:00, 2009.73it/s]
2it [00:00, 1008.12it/s]
2it [00:00, 2006.36it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1002.94it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.90it/s]
1it [00:00, 1003.90it/s]
1it [00:00, 1044.40it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 983.89it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.31it/s]
1it [00:00, 1004.86it/s]
1it [00:00, ?it/s]
1it [00:00, 1043.88it/s]
1it [00:00, 1001.27it/s]
1it [00:00, 997.69it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.38it/s]
1it [00:00, 1004.62it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 489.65it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1005.59it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.38it/s]
1it [00:00, 968.66it/s]
1it [00:00, ?it/s]
1

Successfully created the TFRecord file for  BDG


In [74]:
# image_dir = './Dataset/test/JPEGImages'
# annotations_dir = './Dataset/test/Annotations'
# output_file = './monument-test.tfrecord'
# main(image_dir, annotations_dir, output_file)

In [ ]:
# def xml_to_csv(path):
#     xml_list = []
#     for xml_file in glob.glob(path + '/*.xml'):
#         tree = ET.parse(xml_file)
#         root = tree.getroot()
#         for member in root.findall('object'):
#             class_label = check_class_validity(member[0].text)
#             if class_label is not None:
#                 value = (root.find('filename').text,
#                          int(root.find('size')[0].text),
#                          int(root.find('size')[1].text),
#                          class_label,
#                          int(member[4][0].text),
#                          int(member[4][1].text),
#                          int(member[4][2].text),
#                          int(member[4][3].text)
#                          )
#                 xml_list.append(value)
#     column_name = ['filename', 'width', 'height',
#                    'class', 'xmin', 'ymin', 'xmax', 'ymax']
#     xml_df = pd.DataFrame(xml_list, columns=column_name)
#     return xml_df